<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
# ((x_train, y_train), (x_test, y_test))
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# https://matplotlib.org/stable/tutorials/colors/colormaps.html
# reverse:_r
import matplotlib.pyplot as plt
plt.imshow(x_train[0], cmap="gray")

In [ ]:
import pandas as pd
pd.DataFrame(x_train[0])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
layers = [
    # 784 * 256 + 256(bias)
    Dense(256, activation="relu", input_dim=784),
    # 128 * 10 + 10(bias)
    Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
# 一個神經元(兩個機率): BinaryCrossEntropy
# 多個神經元: pilog(1/qi): CategoricalCrossEntropy
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(loss=CategoricalCrossentropy(),
       metrics=["accuracy"],
       optimizer="adam")

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print(y_train[0])
print(y_train_cat[0])

In [ ]:
x_train_shape = x_train.reshape(-1, 784) / 255.0
x_test_shape = x_test.reshape(-1, 784) / 255.0

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [
   EarlyStopping(patience=5, restore_best_weights=True) 
]
model.fit(x_train_shape,
     y_train_cat,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

In [ ]:
model.evaluate(x_test_shape, y_test_cat)

In [ ]:
from sklearn.metrics import confusion_matrix
# sklearn: predict_proba/predict
# tensorflow: predict/predict_classes
pre = model.predict_classes(x_test_shape)
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
       columns=[str(i) + "(預測)" for i in range(10)],
       index=[str(i) + "(正確)" for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test)[0]
false_img = x_test[idx]
false_pre = pre[idx]
false_label = y_test[idx]
plt.figure(figsize=(14, 42))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    plt.imshow(false_img[i])
    t = "[O]:{}\n[P]:{}".format(false_label[i], false_pre[i])
    plt.title(t)
    plt.axis("off")

In [ ]:
# Pillow
from PIL import Image
# RGB, RGBA, L(gray)
fn = input("輸入檔名:")
img = Image.open(fn).resize((28, 28)).convert("L")
img_np = np.array(img)
img_np_r = img_np.reshape(1, 784)
proba = model.predict(img_np_r)[0]
for i in range(10):
    print(i, "的機率:", proba[i])
ans = model.predict_classes(img_np_r)[0]
print("答案:", ans)
plt.imshow(img_np)